
# Bu soruya cevap arıyoruz: **kernel_size, padding, stride** nasıl seçilir?

Bu notebook, CNN tasarlarken en çok oynanan üç parametreyi **temelden derine** ele alır:

- **kernel_size (K)**: filtrenin görüş alanı
- **stride (S)**: örnekleme / downsample
- **padding (P)**: sınır davranışı + çıktı boyutu kontrolü

🎯 Amaç: “ezber optimum” değil; **probleme göre** tasarım yapmanı sağlayacak mantık + hesap araçları.

---

## İçerik

1. Conv2D çıktı boyutu formülü + hesaplayıcı
2. Kernel seçimi (3×3 neden default?)
3. Stride seçimi (detay ↔ hız)
4. Padding seçimi (SAME/VALID)
5. Receptive Field (RF) + Effective Stride (jump) hesapları
6. FLOPs sezgisi (K ve S maliyeti)
7. Probleme göre reçeteler (classification / detection / segmentation)
8. Mini araçlar ve örnek “stride planı”



## 1) Conv2D çıkış boyutu: önce şekli kontrol et

Giriş: \(H \times W\)

Parametreler:
- kernel: \(K\)
- padding: \(P\)
- stride: \(S\)
- dilation: \(D\)

\[
H_{out}=\left\lfloor \frac{H+2P - D(K-1) - 1}{S} + 1 \right\rfloor
\quad,\quad
W_{out}=\left\lfloor \frac{W+2P - D(K-1) - 1}{S} + 1 \right\rfloor
\]

📌 Tasarımın ilk kuralı:
> “Bu katmandan sonra feature map kaç olacak?” sorusunu **hesaplayabiliyor** ol.


In [1]:

import math

def conv2d_out(H, W, K=3, S=1, P=1, D=1):
    H_out = math.floor((H + 2*P - D*(K-1) - 1)/S + 1)
    W_out = math.floor((W + 2*P - D*(K-1) - 1)/S + 1)
    return H_out, W_out

# Örnekler
examples = [
    (224,224,3,1,1,1),
    (224,224,3,2,1,1),
    (224,224,7,2,3,1),
    (128,128,5,1,2,1),
]
for H,W,K,S,P,D in examples:
    print(f"in={H}x{W}  K={K} S={S} P={P} D={D} -> out={conv2d_out(H,W,K,S,P,D)}")


in=224x224  K=3 S=1 P=1 D=1 -> out=(224, 224)
in=224x224  K=3 S=2 P=1 D=1 -> out=(112, 112)
in=224x224  K=7 S=2 P=3 D=1 -> out=(112, 112)
in=128x128  K=5 S=1 P=2 D=1 -> out=(128, 128)



## 2) Kernel size (K) nasıl seçilir?

Kernel, “lokal bağlam penceresi”dir.

### 2.1) Modern default: **3×3**
Neden?

- Parametre/FLOPs verimli
- Yığılabilir:
  - 2×(3×3) ≈ 5×5 RF
  - 3×(3×3) ≈ 7×7 RF
- Araya nonlinearity girdiği için ifade gücü artar

### 2.2) Büyük kernel (5×5, 7×7) ne zaman?
- Erken katmanda hızlı bağlam toplamak istiyorsan (bazı classification tasarımları)
- Çok düşük çözünürlükte (32×32 gibi) az katmanla bağlam büyütmek istiyorsan
- Large-kernel tasarımları özel olarak hedefliyorsan (compute maliyetiyle)

### 2.3) Büyük kernel yerine alternatifler
- stacked 3×3
- dilation (RF büyütür; “gridding” riski var)
- depthwise separable
- attention/pooling

📌 Pratik kural:
> Şüphedeysen **3×3 ile başla**; büyük kernel ihtiyacını RF analiziyle gerekçelendir.



## 3) Stride (S) nasıl seçilir?

Stride = örnekleme adımı.  
S=2 → H ve W yaklaşık yarıya iner → compute genelde **~4× düşer** (kanallar sabitse).

Burada hep tradeoff var:
- ✅ detay / küçük nesne / lokalizasyon (S küçük)
- ✅ hız / bellek (S büyük)

### Göreve göre:
- **Classification:** erken downsample çoğu zaman tolere edilir, final stride sıklıkla 32×
- **Detection:** küçük obje varsa erken S=2 riskli; tipik hedef stride 8/16/32 feature map’ler
- **Segmentation:** detay kritik; encoder stride’ı 16’da bırakmak yaygın, dilation+decoder kullanılır

📌 Pratik soru:
> En küçük “anlamlı” desen/nesne girişte kaç piksel?  
Bu sayı, erken stride kararını direkt etkiler.



## 4) Padding (P) nasıl seçilir?

Padding iki şeyi kontrol eder:
1) Output shape  
2) Sınır davranışı (border effects)

### SAME vs VALID
- **VALID:** P=0 → boyut küçülür
- **SAME:** stride=1 iken boyutu korur (odd K için)

Odd K’de klasik SAME:
\[
P = \frac{K-1}{2}
\]

Örnek:
- K=3 → P=1
- K=5 → P=2
- K=7 → P=3

📌 Pratik kural:
> Backbone’da çoğu zaman 3×3 + P=1 “default doğru”dur.



## 5) Receptive Field (RF) ve Effective Stride (Jump)

“Probleme göre nasıl hesaplanır?” sorusunun matematik tarafı burada.

Takip ettiğimiz iki büyüklük:
- **RF:** çıkıştaki 1 nokta girişte ne kadar alan görür?
- **J (jump):** çıkışta 1 piksel adım, girişte kaç piksel adım?

Güncelleme:
\[
J_l = J_{l-1}\cdot S_l
\]
\[
RF_l = RF_{l-1} + (K_l-1)\cdot D_l \cdot J_{l-1}
\]
Başlangıç: \(RF_0=1, J_0=1\)

Aşağıdaki araç, verdiğin layer listesinden RF/J trace üretir.


In [2]:

def rf_trace(layers):
    RF, J = 1, 1
    rows = []
    for i, L in enumerate(layers, 1):
        K = L["K"]; S = L["S"]; D = L.get("D", 1)
        RF = RF + (K - 1) * D * J
        J = J * S
        rows.append({"layer": i, "K": K, "S": S, "D": D, "RF": RF, "J": J})
    return rows

layers_early = [{"K":3,"S":2},{"K":3,"S":1},{"K":3,"S":1},{"K":3,"S":2},{"K":3,"S":1}]
layers_late  = [{"K":3,"S":1},{"K":3,"S":1},{"K":3,"S":2},{"K":3,"S":2},{"K":3,"S":1}]

for name, layers in [("Erken downsample", layers_early), ("Geç downsample", layers_late)]:
    print("\n", name)
    for r in rf_trace(layers):
        print(r)



 Erken downsample
{'layer': 1, 'K': 3, 'S': 2, 'D': 1, 'RF': 3, 'J': 2}
{'layer': 2, 'K': 3, 'S': 1, 'D': 1, 'RF': 7, 'J': 2}
{'layer': 3, 'K': 3, 'S': 1, 'D': 1, 'RF': 11, 'J': 2}
{'layer': 4, 'K': 3, 'S': 2, 'D': 1, 'RF': 15, 'J': 4}
{'layer': 5, 'K': 3, 'S': 1, 'D': 1, 'RF': 23, 'J': 4}

 Geç downsample
{'layer': 1, 'K': 3, 'S': 1, 'D': 1, 'RF': 3, 'J': 1}
{'layer': 2, 'K': 3, 'S': 1, 'D': 1, 'RF': 5, 'J': 1}
{'layer': 3, 'K': 3, 'S': 2, 'D': 1, 'RF': 7, 'J': 2}
{'layer': 4, 'K': 3, 'S': 2, 'D': 1, 'RF': 11, 'J': 4}
{'layer': 5, 'K': 3, 'S': 1, 'D': 1, 'RF': 19, 'J': 4}



### 5.1) RF hedefi nasıl konur?

Tek “doğru RF” yok. Hedef:
- görev (classification/detection/seg)
- nesne ölçekleri
- stride seviyeleri (8/16/32)
ile belirlenir.

Pratik yöntem:
1) Giriş boyutu + target stride seviyeleri seç  
2) Bu seviyelerdeki RF’nin **nesne boyutlarını** kapsayıp kapsamadığını kontrol et  
3) Yetmiyorsa: daha derin, dilation, ya da farklı stride yerleşimi

> RF’yi büyütmek kolay; asıl mesele **detayı öldürmeden** büyütmek.



## 6) FLOPs sezgisi: K ve S maliyeti

Yaklaşık Conv FLOPs:

\[
FLOPs \approx 2\cdot H_{out}W_{out}\cdot C_{in}C_{out}\cdot K^2
\]

- K büyürse maliyet \(K^2\) ile artar
- S=2 olursa \(H_{out}W_{out}\) yaklaşık 4× düşer

Aşağıda basit bir FLOPs hesaplayıcı ile kıyas yapıyoruz.


In [3]:

def approx_conv_flops(H, W, Cin, Cout, K=3, S=1, P=1, D=1):
    Hout, Wout = conv2d_out(H, W, K=K, S=S, P=P, D=D)
    return 2 * Hout * Wout * Cin * Cout * (K*K)

H, W = 224, 224
Cin, Cout = 64, 64

for K in [3,5,7]:
    for S in [1,2]:
        P = (K-1)//2  # SAME (odd K)
        fl = approx_conv_flops(H, W, Cin, Cout, K=K, S=S, P=P)
        Hout, Wout = conv2d_out(H, W, K=K, S=S, P=P)
        print(f"K={K} S={S} P={P} -> {Hout}x{Wout} | approx {fl/1e9:.2f} GFLOPs")


K=3 S=1 P=1 -> 224x224 | approx 3.70 GFLOPs
K=3 S=2 P=1 -> 112x112 | approx 0.92 GFLOPs
K=5 S=1 P=2 -> 224x224 | approx 10.28 GFLOPs
K=5 S=2 P=2 -> 112x112 | approx 2.57 GFLOPs
K=7 S=1 P=3 -> 224x224 | approx 20.14 GFLOPs
K=7 S=2 P=3 -> 112x112 | approx 5.04 GFLOPs



## 7) “En optimum” nasıl bulunur?

“Optimum”, tek metrik değildir. Genelde aynı anda:
- accuracy / mAP / mIoU
- latency / FLOPs
- bellek / batch size

Optimum bulma pratikleri:

### 7.1) Constraint tabanlı tasarım (analitik)
- target stride seviyeleri (8/16/32)
- RF ihtiyacı (nesne ölçekleri)
- compute bütçesi

→ stride planı + kanal planı kurulur.

### 7.2) Controlled ablation
- K sabit (genelde 3)
- stride yerleşimi varyasyonları
- downsample yöntemi varyasyonları (stride conv vs pool vs blurpool)
- dilation varyasyonları

Sonra Pareto bakarsın: “daha hızlı ama az daha kötü” vs “daha yavaş ama daha iyi”.

📌 Gerçek dünya:
> Default doğru ile başla → küçük kontrollü oynamalarla iyileştir.



## 8) Probleme göre şablonlar + stride planı çıktısı

Aşağıdaki hücre, basit bir stride planından **total stride** ve yaklaşık **feature map boyutları** çıkarır.

- Classification (224×224): final stride 32 hedefi yaygın
- Detection (640×640): P3/P4/P5 için stride 8/16/32
- Segmentation: stride 16’da kalmak yaygın (detay için)

Not: Bu çıktı “plan”dır; gerçek ağda conv formülüyle doğrulamak gerekir.


In [4]:

def stage_shapes(H, W, strides):
    out = []
    tot = 1
    h, w = H, W
    for i, s in enumerate(strides, 1):
        tot *= s
        # kaba /s (conv formülü farklı olabilir; burada plan sezgisi)
        h = math.floor((h + s - 1)/s)
        w = math.floor((w + s - 1)/s)
        out.append((f"stage{i}", h, w, tot))
    return out

print("Classification 224×224, strides=[2,2,2,2,2] (total 32×)")
for row in stage_shapes(224,224,[2,2,2,2,2]):
    print(row)

print("\nDetection 640×640, strides=[2,2,2,2,2] (P3=8, P4=16, P5=32 seçersin)")
for row in stage_shapes(640,640,[2,2,2,2,2]):
    print(row)

print("\nSegmentation örnek: total stride 16× hedefi için strides=[2,2,2,2]")
for row in stage_shapes(512,512,[2,2,2,2]):
    print(row)


Classification 224×224, strides=[2,2,2,2,2] (total 32×)
('stage1', 112, 112, 2)
('stage2', 56, 56, 4)
('stage3', 28, 28, 8)
('stage4', 14, 14, 16)
('stage5', 7, 7, 32)

Detection 640×640, strides=[2,2,2,2,2] (P3=8, P4=16, P5=32 seçersin)
('stage1', 320, 320, 2)
('stage2', 160, 160, 4)
('stage3', 80, 80, 8)
('stage4', 40, 40, 16)
('stage5', 20, 20, 32)

Segmentation örnek: total stride 16× hedefi için strides=[2,2,2,2]
('stage1', 256, 256, 2)
('stage2', 128, 128, 4)
('stage3', 64, 64, 8)
('stage4', 32, 32, 16)



## 9) Mühendislik kontrol listesi (kısa ama kritik)

Bir katman/stage tasarlarken:

1. **Shape**: H×W kaç olacak? (conv formülüyle doğrula)
2. **Total stride**: Bu seviyenin girişe göre stride’ı kaç? (8/16/32)
3. **RF**: Bu seviyedeki RF, hedef nesne ölçeklerini kapsıyor mu?
4. **Compute**: Bu seviyede maliyet çok mu yüksek? (özellikle yüksek çözünürlükte)
5. **Aliasing**: stride=2 yaptığın yerde sinyali “kırpıyor” musun? (gerekirse downsample yöntemi değiştir)
6. **Görev uyumu**: classification/detection/seg için uygun mu?

---

## Son söz

> Kernel/padding/stride seçimi, “tablodan değer almak” değil,  
> shape + RF + compute üçlüsünü hesaplayıp **kontrollü ablation** ile doğrulamaktır.
